## Cell 1: Configuration

In [1]:
# Configuration settings for the Library Management System
import os
import json
from datetime import datetime, timedelta

# Configuration class
class Config:
    # Database file paths
    STUDENTS_DB_PATH = "students_db.json"
    ADMINS_DB_PATH = "admins_db.json"
    BOOKS_DB_PATH = "books_db.json"
    
    # Borrowing settings
    DEFAULT_BORROW_DAYS = 14
    
    # Sample data for initialization
    SAMPLE_STUDENTS = {
        "student1": {
            "password": "pass1",
            "name": "John Doe",
            "borrowed_books": []
        },
        "student2": {
            "password": "pass2",
            "name": "Jane Smith",
            "borrowed_books": []
        }
    }
    
    SAMPLE_ADMINS = {
        "admin1": {
            "password": "admin1",
            "name": "Library Admin"
        },
        "admin2": {
            "password": "admin2",
            "name": "System Admin"
        }
    }
    
    SAMPLE_BOOKS = {
        "1": {
            "title": "Python Programming",
            "author": "John Smith",
            "status": "available",
            "borrowed_by": None,
            "due_date": None
        },
        "2": {
            "title": "Data Science Essentials",
            "author": "Emily Johnson",
            "status": "available",
            "borrowed_by": None,
            "due_date": None
        },
        "3": {
            "title": "Machine Learning Basics",
            "author": "Michael Chen",
            "status": "available",
            "borrowed_by": None,
            "due_date": None
        },
        "4": {
            "title": "Algorithms and Data Structures",
            "author": "Sarah Williams",
            "status": "available",
            "borrowed_by": None,
            "due_date": None
        },
        "5": {
            "title": "Web Development with Flask",
            "author": "David Brown",
            "status": "available",
            "borrowed_by": None,
            "due_date": None
        }
    }

print("Configuration loaded successfully!")

Configuration loaded successfully!


## Cell 2: Core Models

In [2]:
# Core model classes for the Library Management System
from datetime import datetime, timedelta

class Book:
    """Book class representing a library book"""
    def __init__(self, book_id, title, author, status="available", borrowed_by=None, due_date=None):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.status = status
        self.borrowed_by = borrowed_by
        self.due_date = due_date
    
    def to_dict(self):
        """Convert book object to dictionary for JSON storage"""
        return {
            "title": self.title,
            "author": self.author,
            "status": self.status,
            "borrowed_by": self.borrowed_by,
            "due_date": self.due_date
        }
    
    @classmethod
    def from_dict(cls, book_id, book_dict):
        """Create a Book object from dictionary data"""
        return cls(
            book_id=book_id,
            title=book_dict["title"],
            author=book_dict["author"],
            status=book_dict["status"],
            borrowed_by=book_dict["borrowed_by"],
            due_date=book_dict["due_date"]
        )


class User:
    """Base user class"""
    def __init__(self, username, password, name):
        self.username = username
        self.password = password
        self.name = name
    
    def authenticate(self, password):
        """Authenticate user with password"""
        return self.password == password


class Student(User):
    """Student user class"""
    def __init__(self, username, password, name, borrowed_books=None):
        super().__init__(username, password, name)
        self.borrowed_books = borrowed_books or []
    
    def to_dict(self):
        """Convert student object to dictionary for JSON storage"""
        return {
            "password": self.password,
            "name": self.name,
            "borrowed_books": self.borrowed_books
        }
    
    @classmethod
    def from_dict(cls, username, student_dict):
        """Create a Student object from dictionary data"""
        return cls(
            username=username,
            password=student_dict["password"],
            name=student_dict["name"],
            borrowed_books=student_dict["borrowed_books"]
        )


class Admin(User):
    """Admin user class"""
    def to_dict(self):
        """Convert admin object to dictionary for JSON storage"""
        return {
            "password": self.password,
            "name": self.name
        }
    
    @classmethod
    def from_dict(cls, username, admin_dict):
        """Create an Admin object from dictionary data"""
        return cls(
            username=username,
            password=admin_dict["password"],
            name=admin_dict["name"]
        )

print("Core models defined successfully!")

Core models defined successfully!


## Cell 3: Database Manager

In [3]:
# Database Manager for handling data persistence
import os
import json
from datetime import datetime

class DatabaseManager:
    """Handles all database operations for the Library Management System"""
    
    def __init__(self, config):
        """Initialize with configuration"""
        self.config = config
        self.init_databases()
    
    def init_databases(self):
        """Initialize database files if they don't exist"""
        # Create students database
        if not os.path.exists(self.config.STUDENTS_DB_PATH):
            with open(self.config.STUDENTS_DB_PATH, 'w') as f:
                json.dump(self.config.SAMPLE_STUDENTS, f, indent=4)
        
        # Create admins database
        if not os.path.exists(self.config.ADMINS_DB_PATH):
            with open(self.config.ADMINS_DB_PATH, 'w') as f:
                json.dump(self.config.SAMPLE_ADMINS, f, indent=4)
        
        # Create books database
        if not os.path.exists(self.config.BOOKS_DB_PATH):
            with open(self.config.BOOKS_DB_PATH, 'w') as f:
                json.dump(self.config.SAMPLE_BOOKS, f, indent=4)
    
    # Student operations
    def get_student(self, username):
        """Get student data by username"""
        students_data = self._load_data(self.config.STUDENTS_DB_PATH)
        if username in students_data:
            return Student.from_dict(username, students_data[username])
        return None
    
    def authenticate_student(self, username, password):
        """Authenticate student credentials"""
        student = self.get_student(username)
        if student and student.authenticate(password):
            return student
        return None
    
    def add_student(self, student):
        """Add or update student in database"""
        students_data = self._load_data(self.config.STUDENTS_DB_PATH)
        students_data[student.username] = student.to_dict()
        self._save_data(self.config.STUDENTS_DB_PATH, students_data)
    
    def remove_student(self, username):
        """Remove student from database"""
        students_data = self._load_data(self.config.STUDENTS_DB_PATH)
        if username in students_data:
            del students_data[username]
            self._save_data(self.config.STUDENTS_DB_PATH, students_data)
            return True
        return False
    
    def get_all_students(self):
        """Get all students from database"""
        students_data = self._load_data(self.config.STUDENTS_DB_PATH)
        return {username: Student.from_dict(username, data) 
                for username, data in students_data.items()}
    
    # Admin operations
    def get_admin(self, username):
        """Get admin data by username"""
        admins_data = self._load_data(self.config.ADMINS_DB_PATH)
        if username in admins_data:
            return Admin.from_dict(username, admins_data[username])
        return None
    
    def authenticate_admin(self, username, password):
        """Authenticate admin credentials"""
        admin = self.get_admin(username)
        if admin and admin.authenticate(password):
            return admin
        return None
    
    # Book operations
    def get_book(self, book_id):
        """Get book data by ID"""
        books_data = self._load_data(self.config.BOOKS_DB_PATH)
        if book_id in books_data:
            return Book.from_dict(book_id, books_data[book_id])
        return None
    
    def add_book(self, book):
        """Add or update book in database"""
        books_data = self._load_data(self.config.BOOKS_DB_PATH)
        books_data[book.book_id] = book.to_dict()
        self._save_data(self.config.BOOKS_DB_PATH, books_data)
    
    def remove_book(self, book_id):
        """Remove book from database"""
        books_data = self._load_data(self.config.BOOKS_DB_PATH)
        if book_id in books_data:
            del books_data[book_id]
            self._save_data(self.config.BOOKS_DB_PATH, books_data)
            return True
        return False
    
    def get_all_books(self):
        """Get all books from database"""
        books_data = self._load_data(self.config.BOOKS_DB_PATH)
        return {book_id: Book.from_dict(book_id, data) 
                for book_id, data in books_data.items()}
    
    def get_available_books(self):
        """Get all available books"""
        all_books = self.get_all_books()
        return {book_id: book for book_id, book in all_books.items() 
                if book.status == "available"}
    
    # Borrowing operations
    def borrow_book(self, book_id, student_username):
        """Record book borrowing"""
        book = self.get_book(book_id)
        student = self.get_student(student_username)
        
        if not book or not student:
            return False
        
        if book.status != "available":
            return False
        
        # Update book status
        book.status = "borrowed"
        book.borrowed_by = student_username
        book.due_date = (datetime.now() + timedelta(days=self.config.DEFAULT_BORROW_DAYS)).strftime("%Y-%m-%d")
        self.add_book(book)
        
        # Update student's borrowed books
        student.borrowed_books.append(book_id)
        self.add_student(student)
        
        return True
    
    def return_book(self, book_id, student_username):
        """Process book return"""
        book = self.get_book(book_id)
        student = self.get_student(student_username)
        
        if not book or not student:
            return False
        
        if book.status != "borrowed" or book.borrowed_by != student_username:
            return False
        
        # Update book status
        book.status = "available"
        book.borrowed_by = None
        book.due_date = None
        self.add_book(book)
        
        # Update student's borrowed books
        if book_id in student.borrowed_books:
            student.borrowed_books.remove(book_id)
            self.add_student(student)
        
        return True
    
    # Helper methods
    def _load_data(self, file_path):
        """Load data from JSON file"""
        with open(file_path, 'r') as f:
            return json.load(f)
    
    def _save_data(self, file_path, data):
        """Save data to JSON file"""
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

# Import the required classes and create instance for testing
from datetime import datetime, timedelta

# Create an instance of the database manager for testing
db_manager = DatabaseManager(Config)
print("Database Manager initialized successfully!")

Database Manager initialized successfully!


## Cell 4: UI Components

In [4]:
# UI Components for the Library Management System
import ipywidgets as widgets
from IPython.display import display, clear_output

class UIComponents:
    """UI component builder for Library Management System"""
    
    @staticmethod
    def create_button(description, button_style="primary", width="200px", on_click=None):
        """Create a styled button"""
        btn = widgets.Button(
            description=description,
            button_style=button_style,
            layout=widgets.Layout(width=width)
        )
        if on_click:
            btn.on_click(on_click)
        return btn
    
    @staticmethod
    def create_text_input(description, password=False, width="300px"):
        """Create a text input field"""
        if password:
            return widgets.Password(
                description=description,
                layout=widgets.Layout(width=width)
            )
        else:
            return widgets.Text(
                description=description,
                layout=widgets.Layout(width=width)
            )
    
    @staticmethod
    def create_message(message, message_type="info"):
        """Create a styled message display"""
        style = {
            "info": "background-color: #d9edf7; color: #31708f; padding: 10px; border-radius: 5px;",
            "success": "background-color: #dff0d8; color: #3c763d; padding: 10px; border-radius: 5px;",
            "warning": "background-color: #fcf8e3; color: #8a6d3b; padding: 10px; border-radius: 5px;",
            "error": "background-color: #f2dede; color: #a94442; padding: 10px; border-radius: 5px;"
        }
        
        return widgets.HTML(f"<div style='{style[message_type]}'>{message}</div>")
    
    @staticmethod
    def create_heading(text, level=1):
        """Create HTML heading"""
        return widgets.HTML(f"<h{level}>{text}</h{level}>")
    
    @staticmethod
    def create_paragraph(text):
        """Create HTML paragraph"""
        return widgets.HTML(f"<p>{text}</p>")
    
    @staticmethod
    def create_book_item(book, book_id, with_actions=True, action_text="Borrow", action_style="success", on_action=None):
        """Create a UI component for displaying a book"""
        book_html = (
            f"<div style='margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>"
            f"<strong>ID:</strong> {book_id}<br>"
            f"<strong>Title:</strong> {book.title}<br>"
            f"<strong>Author:</strong> {book.author}"
        )
        
        if hasattr(book, 'status') and book.status != "available":
            book_html += f"<br><strong>Status:</strong> {book.status}"
        
        if hasattr(book, 'due_date') and book.due_date:
            book_html += f"<br><strong>Due Date:</strong> {book.due_date}"
            
        book_html += "</div>"
        
        book_widget = widgets.HTML(book_html)
        
        if with_actions and on_action:
            btn = UIComponents.create_button(
                f"{action_text} '{book.title}'", 
                button_style=action_style,
                width="auto"
            )
            btn.on_click(on_action)
            return widgets.VBox([book_widget, btn])
        
        return book_widget
    
    @staticmethod
    def create_student_item(student, username, with_actions=True, view_handler=None, remove_handler=None):
        """Create a UI component for displaying a student"""
        student_html = (
            f"<div style='margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>"
            f"<strong>Username:</strong> {username}<br>"
            f"<strong>Name:</strong> {student.name}<br>"
            f"<strong>Books Borrowed:</strong> {len(student.borrowed_books)}"
            f"</div>"
        )
        
        student_widget = widgets.HTML(student_html)
        
        if with_actions:
            buttons = []
            
            if view_handler:
                view_btn = UIComponents.create_button(
                    "View Details", 
                    button_style="info",
                    width="auto"
                )
                view_btn.on_click(view_handler)
                buttons.append(view_btn)
            
            if remove_handler:
                remove_btn = UIComponents.create_button(
                    "Remove Student", 
                    button_style="danger",
                    width="auto"
                )
                remove_btn.on_click(remove_handler)
                buttons.append(remove_btn)
                
            return widgets.VBox([
                student_widget, 
                widgets.HBox(buttons)
            ])
        
        return student_widget

print("UI Components initialized successfully!")

UI Components initialized successfully!


## Cell 5: Main Application - Part 1 (Core Functionality)

In [5]:
# Main Application class for Library Management System - Core functionality
import ipywidgets as widgets
from IPython.display import display, clear_output

class LibrarySystem:
    """Main Library Management System application"""
    
    def __init__(self, config, db_manager, ui):
        """Initialize the library system"""
        self.config = config
        self.db = db_manager
        self.ui = ui
        
        # Current session data
        self.current_user = None
        self.user_type = None
        
        # Main display area
        self.main_output = widgets.Output()
        
        # Main UI container
        self.main_ui = None
    
    def start(self):
        """Start the library system"""
        self.setup_main_ui()
        display(self.main_ui)
    
    def setup_main_ui(self):
        """Set up the initial UI"""
        self.main_ui = widgets.VBox([
            self.ui.create_heading("Library Management System"),
            self.ui.create_heading("Please select an option to continue:", level=3),
            widgets.HBox([
                self.ui.create_button("Login as Student", "info", on_click=self.show_student_login),
                self.ui.create_button("Login as Admin", "danger", on_click=self.show_admin_login),
                self.ui.create_button("Sign Up (Student)", "success", on_click=self.show_student_signup)
            ]),
            self.main_output
        ])
    
    def clear_main_output(self):
        """Clear the main output widget"""
        self.main_output.clear_output()
    
    def show_message(self, message, message_type="info"):
        """Display a message in the output area"""
        with self.main_output:
            display(self.ui.create_message(message, message_type))
    
    def logout(self):
        """Log out the current user"""
        self.current_user = None
        self.user_type = None
        self.clear_main_output()
        self.setup_main_ui()
        display(self.main_ui)
    
    # Authentication methods
    def show_student_login(self, btn):
        """Show student login form"""
        self.clear_main_output()
        
        username_widget = self.ui.create_text_input("Username:")
        password_widget = self.ui.create_text_input("Password:", password=True)
        login_btn = self.ui.create_button("Login", "primary")
        back_btn = self.ui.create_button("Back", "warning")
        
        login_form = widgets.VBox([
            self.ui.create_heading("Student Login", level=2),
            username_widget,
            password_widget,
            widgets.HBox([login_btn, back_btn])
        ])
        
        def login_handler(btn):
            username = username_widget.value
            password = password_widget.value
            
            if not username or not password:
                self.show_message("Please enter both username and password!", "error")
                return
            
            student = self.db.authenticate_student(username, password)
            if student:
                self.current_user = student
                self.user_type = "student"
                self.clear_main_output()
                self.show_student_dashboard()
            else:
                self.show_message("Invalid username or password!", "error")
        
        login_btn.on_click(login_handler)
        back_btn.on_click(lambda btn: self.start())
        
        with self.main_output:
            display(login_form)
    
    def show_admin_login(self, btn):
        """Show admin login form"""
        self.clear_main_output()
        
        username_widget = self.ui.create_text_input("Username:")
        password_widget = self.ui.create_text_input("Password:", password=True)
        login_btn = self.ui.create_button("Login", "primary")
        back_btn = self.ui.create_button("Back", "warning")
        
        login_form = widgets.VBox([
            self.ui.create_heading("Admin Login", level=2),
            username_widget,
            password_widget,
            widgets.HBox([login_btn, back_btn])
        ])
        
        def login_handler(btn):
            username = username_widget.value
            password = password_widget.value
            
            if not username or not password:
                self.show_message("Please enter both username and password!", "error")
                return
            
            admin = self.db.authenticate_admin(username, password)
            if admin:
                self.current_user = admin
                self.user_type = "admin"
                self.clear_main_output()
                self.show_admin_dashboard()
            else:
                self.show_message("Invalid username or password!", "error")
        
        login_btn.on_click(login_handler)
        back_btn.on_click(lambda btn: self.start())
        
        with self.main_output:
            display(login_form)
    
    def show_student_signup(self, btn):
        """Show student signup form"""
        self.clear_main_output()
        
        username_widget = self.ui.create_text_input("Username:")
        password_widget = self.ui.create_text_input("Password:", password=True)
        name_widget = self.ui.create_text_input("Full Name:")
        signup_btn = self.ui.create_button("Sign Up", "primary")
        back_btn = self.ui.create_button("Back", "warning")
        
        signup_form = widgets.VBox([
            self.ui.create_heading("Student Sign Up", level=2),
            username_widget,
            password_widget,
            name_widget,
            widgets.HBox([signup_btn, back_btn])
        ])
        
        def signup_handler(btn):
            username = username_widget.value
            password = password_widget.value
            name = name_widget.value
            
            if not username or not password or not name:
                self.show_message("Please fill in all fields!", "error")
                return
            
            # Check if username already exists
            if self.db.get_student(username):
                self.show_message("Username already exists!", "error")
                return
            
            # Create new student
            student = Student(username, password, name)
            self.db.add_student(student)
            
            self.show_message("Account created successfully! Please login.", "success")
            # Wait a bit and go back to main menu
            import time
            time.sleep(2)
            self.start()
        
        signup_btn.on_click(signup_handler)
        back_btn.on_click(lambda btn: self.start())
        
        with self.main_output:
            display(signup_form)

print("LibrarySystem core functionality initialized successfully!")

LibrarySystem core functionality initialized successfully!


## Cell 6: Main Application - Part 2 (Student Features)

In [7]:
# Main Application class extension - Student features
class LibrarySystem(LibrarySystem):  # Extending the class from previous cell
    
    def show_student_dashboard(self):
        """Display student dashboard"""
        if not self.current_user or self.user_type != "student":
            self.show_message("You need to login as a student first!", "error")
            return
        
        self.clear_main_output()
        
        browse_books_btn = self.ui.create_button("Browse Books", "primary")
        view_borrowed_btn = self.ui.create_button("My Borrowed Books", "info")
        logout_btn = self.ui.create_button("Logout", "danger")
        
        dashboard_output = widgets.Output()
        
        student_dashboard = widgets.VBox([
            self.ui.create_heading(f"Welcome, {self.current_user.name}!"),
            self.ui.create_paragraph(f"You currently have {len(self.current_user.borrowed_books)} books borrowed."),
            widgets.HBox([browse_books_btn, view_borrowed_btn, logout_btn]),
            dashboard_output
        ])
        
        def browse_books_handler(btn):
            """Handle browse books button click"""
            dashboard_output.clear_output()
            self.show_available_books_for_student(dashboard_output)
            
        def view_borrowed_handler(btn):
            """Handle view borrowed books button click"""
            dashboard_output.clear_output()
            self.show_borrowed_books(dashboard_output)
        
        browse_books_btn.on_click(browse_books_handler)
        view_borrowed_btn.on_click(view_borrowed_handler)
        logout_btn.on_click(lambda btn: self.logout())
        
        with self.main_output:
            display(student_dashboard)
    
    def show_available_books_for_student(self, output_widget):
        """Display available books for students to borrow"""
        available_books = self.db.get_available_books()
        
        if not available_books:
            with output_widget:
                display(self.ui.create_paragraph("<i>No books available for borrowing at the moment.</i>"))
            return
        
        book_items = []
        for book_id, book in available_books.items():
            # Create a handler function for each book
            def create_borrow_handler(b_id):
                def handler(btn):
                    self.borrow_book(b_id)
                return handler
            
            book_item = self.ui.create_book_item(
                book, 
                book_id, 
                action_text="Borrow",
                on_action=create_borrow_handler(book_id)
            )
            book_items.append(book_item)
        
        with output_widget:
            display(self.ui.create_heading("Available Books", level=3))
            display(widgets.VBox(book_items))
    
    def borrow_book(self, book_id):
        """Allow student to borrow a book"""
        if not self.current_user or self.user_type != "student":
            self.show_message("You need to login as a student first!", "error")
            return
        
        if self.db.borrow_book(book_id, self.current_user.username):
            # Refresh current user data
            self.current_user = self.db.get_student(self.current_user.username)
            book = self.db.get_book(book_id)
            self.show_message(f"Successfully borrowed '{book.title}'", "success")
            
            # Refresh the student dashboard
            import time
            time.sleep(1)
            self.show_student_dashboard()
        else:
            self.show_message("Error: Book is not available for borrowing!", "error")
    
    def show_borrowed_books(self, output_widget):
        """Display books borrowed by the current student"""
        if not self.current_user or self.user_type != "student":
            self.show_message("You need to login as a student first!", "error")
            return
        
        borrowed_book_ids = self.current_user.borrowed_books
        
        if not borrowed_book_ids:
            with output_widget:
                display(self.ui.create_paragraph("<i>You haven't borrowed any books yet.</i>"))
            return
        
        book_items = []
        for book_id in borrowed_book_ids:
            book = self.db.get_book(book_id)
            if not book:
                continue
                
            # Create a handler function for each book
            def create_return_handler(b_id):
                def handler(btn):
                    self.return_book(b_id)
                return handler
            
            book_item = self.ui.create_book_item(
                book, 
                book_id, 
                action_text="Return",
                action_style="warning",
                on_action=create_return_handler(book_id)
            )
            book_items.append(book_item)
        
        with output_widget:
            display(self.ui.create_heading("My Borrowed Books", level=3))
            display(widgets.VBox(book_items))
    
    def return_book(self, book_id):
        """Allow student to return a borrowed book"""
        if not self.current_user or self.user_type != "student":
            self.show_message("You need to login as a student first!", "error")
            return
        
        if self.db.return_book(book_id, self.current_user.username):
            # Refresh current user data
            self.current_user = self.db.get_student(self.current_user.username)
            book = self.db.get_book(book_id)
            self.show_message(f"Successfully returned '{book.title}'", "success")
            
            # Refresh the student dashboard
            import time
            time.sleep(1)
            self.show_student_dashboard()
        else:
            self.show_message("Error: Unable to return this book!", "error")

print("Student features initialized successfully!")

Student features initialized successfully!


## Cell 7: Main Application - Part 3 (Admin Features)

In [9]:
# Main Application class extension - Admin features
class LibrarySystem(LibrarySystem):  # Extending the class from previous cells
    
    def show_admin_dashboard(self):
        """Display admin dashboard"""
        if not self.current_user or self.user_type != "admin":
            self.show_message("You need to login as an admin first!", "error")
            return
        
        self.clear_main_output()
        
        manage_books_btn = self.ui.create_button("Manage Books", "primary")
        manage_students_btn = self.ui.create_button("Manage Students", "info")
        logout_btn = self.ui.create_button("Logout", "danger")
        
        dashboard_output = widgets.Output()
        
        admin_dashboard = widgets.VBox([
            self.ui.create_heading(f"Admin Dashboard - {self.current_user.name}"),
            widgets.HBox([manage_books_btn, manage_students_btn, logout_btn]),
            dashboard_output
        ])
        
        def manage_books_handler(btn):
            """Handle manage books button click"""
            dashboard_output.clear_output()
            self.show_books_management(dashboard_output)
            
        def manage_students_handler(btn):
            """Handle manage students button click"""
            dashboard_output.clear_output()
            self.show_students_management(dashboard_output)
        
        manage_books_btn.on_click(manage_books_handler)
        manage_students_btn.on_click(manage_students_handler)
        logout_btn.on_click(lambda btn: self.logout())
        
        with self.main_output:
            display(admin_dashboard)
    
    def show_books_management(self, output_widget):
        """Display books management interface for admin"""
        if not self.current_user or self.user_type != "admin":
            self.show_message("You need to login as an admin first!", "error")
            return
            
        # Form to add new book
        title_widget = self.ui.create_text_input("Title:")
        author_widget = self.ui.create_text_input("Author:")
        add_book_btn = self.ui.create_button("Add Book", "success")
        
        def add_book_handler(btn):
            title = title_widget.value
            author = author_widget.value
            
            if not title or not author:
                self.show_message("Please fill in all fields!", "error")
                return
            
            all_books = self.db.get_all_books()
            # Generate new ID
            new_id = str(max([int(id) for id in all_books.keys()]) + 1) if all_books else "1"
            
            # Create new book
            new_book = Book(new_id, title, author)
            self.db.add_book(new_book)
            
            self.show_message(f"Book '{title}' added successfully!", "success")
            
            # Refresh the book management view
            import time
            time.sleep(1)
            self.show_books_management(output_widget)
            
        add_book_btn.on_click(add_book_handler)
        
        # Display all books with remove buttons
        all_books = self.db.get_all_books()
        book_items = []
        
        for book_id, book in all_books.items():
            # Create a handler function for each book
            def create_remove_handler(b_id):
                def handler(btn):
                    self.remove_book(b_id)
                return handler
            
            status_info = f"<br><strong>Status:</strong> {book.status}"
            if book.borrowed_by:
                status_info += f"<br><strong>Borrowed By:</strong> {book.borrowed_by}"
            if book.due_date:
                status_info += f"<br><strong>Due Date:</strong> {book.due_date}"
                
            book_html = (
                f"<div style='margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>"
                f"<strong>ID:</strong> {book_id}<br>"
                f"<strong>Title:</strong> {book.title}<br>"
                f"<strong>Author:</strong> {book.author}"
                f"{status_info}"
                f"</div>"
            )
            
            remove_btn = self.ui.create_button(
                f"Remove '{book.title}'", 
                button_style="danger",
                width="auto"
            )
            
            remove_btn.on_click(create_remove_handler(book_id))
            book_items.append(widgets.VBox([widgets.HTML(book_html), remove_btn]))
        
        with output_widget:
            display(widgets.VBox([
                self.ui.create_heading("Add New Book", level=3),
                title_widget,
                author_widget,
                add_book_btn,
                self.ui.create_heading("Existing Books", level=3),
                widgets.VBox(book_items) if book_items else self.ui.create_paragraph("<i>No books in the library yet.</i>")
            ]))
    
    def remove_book(self, book_id):
        """Allow admin to remove a book"""
        if not self.current_user or self.user_type != "admin":
            self.show_message("You need to login as an admin first!", "error")
            return
        
        book = self.db.get_book(book_id)
        if not book:
            self.show_message("Error: Book not found!", "error")
            return
            
        # If the book is borrowed, process returns
        if book.status == "borrowed" and book.borrowed_by:
            student = self.db.get_student(book.borrowed_by)
            if student and book_id in student.borrowed_books:
                student.borrowed_books.remove(book_id)
                self.db.add_student(student)
        
        # Remove the book
        if self.db.remove_book(book_id):
            self.show_message(f"Book '{book.title}' removed successfully!", "success")
            
            # Refresh the admin dashboard
            import time
            time.sleep(1)
            self.show_admin_dashboard()
        else:
            self.show_message("Error: Failed to remove book!", "error")
    
    def show_students_management(self, output_widget):
        """Display students management interface for admin"""
        if not self.current_user or self.user_type != "admin":
            self.show_message("You need to login as an admin first!", "error")
            return
            
        # Get all students
        all_students = self.db.get_all_students()
        student_items = []
        
        for username, student in all_students.items():
            # Create handler functions for each student
            def create_view_handler(u_name):
                def handler(btn):
                    self.view_student_details(u_name, output_widget)
                return handler
            
            def create_remove_handler(u_name):
                def handler(btn):
                    self.remove_student(u_name)
                return handler
            
            student_item = self.ui.create_student_item(
                student, 
                username,
                view_handler=create_view_handler(username),
                remove_handler=create_remove_handler(username)
            )
            student_items.append(student_item)
        
        with output_widget:
            display(self.ui.create_heading("Registered Students", level=3))
            display(widgets.VBox(student_items) if student_items else self.ui.create_paragraph("<i>No students registered yet.</i>"))
    
    def view_student_details(self, username, output_widget):
        """Display detailed information about a student"""
        if not self.current_user or self.user_type != "admin":
            self.show_message("You need to login as an admin first!", "error")
            return
            
        student = self.db.get_student(username)
        if not student:
            self.show_message("Student not found!", "error")
            return
            
        output_widget.clear_output()
        
        # Get borrowed books details
        borrowed_books = []
        for book_id in student.borrowed_books:
            book = self.db.get_book(book_id)
            if book:
                borrowed_books.append({
                    "id": book_id,
                    "title": book.title,
                    "author": book.author,
                    "due_date": book.due_date
                })
        
        # Create book displays
        book_items = []
        for book in borrowed_books:
            book_html = (
                f"<div style='margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>"
                f"<strong>ID:</strong> {book['id']}<br>"
                f"<strong>Title:</strong> {book['title']}<br>"
                f"<strong>Author:</strong> {book['author']}<br>"
                f"<strong>Due Date:</strong> {book['due_date']}"
                f"</div>"
            )
            book_items.append(widgets.HTML(book_html))
        
        back_btn = self.ui.create_button("Back to Student List", "warning")
        back_btn.on_click(lambda btn: self.show_students_management(output_widget))
        
        with output_widget:
            display(widgets.VBox([
                self.ui.create_heading(f"Student Details: {student.name} ({username})", level=2),
                self.ui.create_heading("Borrowed Books", level=3),
                widgets.VBox(book_items) if book_items else self.ui.create_paragraph("<i>No books currently borrowed.</i>"),
                back_btn
            ]))
    
    def remove_student(self, username):
        """Allow admin to remove a student"""
        if not self.current_user or self.user_type != "admin":
            self.show_message("You need to login as an admin first!", "error")
            return
            
        student = self.db.get_student(username)
        if not student:
            self.show_message("Student not found!", "error")
            return
            
        # Process returns for all borrowed books
        books_data = self.db.get_all_books()
        for book_id in student.borrowed_books:
            book = self.db.get_book(book_id)
            if book and book.status == "borrowed":
                book.status = "available"
                book.borrowed_by = None
                book.due_date = None
                self.db.add_book(book)
        
        # Remove the student
        if self.db.remove_student(username):
            self.show_message(f"Student '{student.name}' removed successfully!", "success")
            
            # Refresh the admin dashboard
            import time
            time.sleep(1)
            self.show_admin_dashboard()
        else:
            self.show_message("Error: Failed to remove student!", "error")

print("Admin features initialized successfully!")

Admin features initialized successfully!


## Cell 8: Initialization and Run

In [11]:
# Initialize and run the Library Management System
from IPython.display import display, clear_output
import ipywidgets as widgets

# Create an instance of the components
config = Config
db_manager = DatabaseManager(config)
ui_components = UIComponents

# Create and start the Library System
library_system = LibrarySystem(config, db_manager, ui_components)

# Display usage information
print("Welcome to the Library Management System!")
print("Credentials for testing:")
print("Student: username='student1', password='pass1'")
print("Admin: username='admin1', password='admin1'")
print("\nThe system will create JSON files in the current directory for storing data.")

# Start the application
library_system.start()

Welcome to the Library Management System!
Credentials for testing:
Student: username='student1', password='pass1'
Admin: username='admin1', password='admin1'

The system will create JSON files in the current directory for storing data.
